In [100]:
from bs4 import BeautifulSoup
import requests
from functools import lru_cache
import re

In [8]:
page = requests.get('https://www.saq.com/en/12073995')

In [9]:
soup = BeautifulSoup(page.content)


In [10]:
soup.title

<title>19 Crimes Shiraz/Grenache/Mataro | Product page | SAQ.COM</title>

In [19]:
soup.find_all('stock_label_container')

[]

In [47]:
HEADER = {'Host': 'www.saq.com',
'Referer': 'https://www.saq.com/en/12073995',
'Host': 'www.saq.com',
'Accept': 'application/json',
'X-Requested-With': 'XMLHttpRequest'}

def retrieve_stock(product_id):
    query = f"https://www.saq.com/en/saq_inventory/product/internal/id/{product_id}/?_=1585498276470"
    response = requests.get(query, headers=HEADER)
    return response.json()
    

In [52]:
retrieve_stock(305444)

{'stock': 156, 'source': 1}

In [53]:
# some bottles don't seem to be queryable under their product code, instead we can fetch it from the pricebox

In [55]:
page = requests.get('https://www.saq.com/en/14376324')
soup = BeautifulSoup(page.content)

In [72]:
ret = soup.find("div", {'data-product-id':'*'})

In [73]:
ret

In [77]:
ret = soup.find('div', {'class':'price-box'})

In [86]:
product_id = int(ret.attrs['data-product-id'])

In [94]:
@lru_cache(maxsize=512)
def fetch_alternative_product_id(original_product_id):
    page = requests.get(f'https://www.saq.com/en/{original_product_id}')
    soup = BeautifulSoup(page.content)
    ret = soup.find('div', {'class':'price-box'})
    product_id = int(ret.attrs['data-product-id'])
    return product_id

In [95]:
original = 14207918
alternative=fetch_alternative_product_id(original)

In [96]:
retrieve_stock(alternative)

{'stock': 8, 'source': 1}

In [98]:
def get_stock_from_webpage(url):
    product_regex = '1{1}\d{7}'

    # Parse product ID out of webpage
    product_id = re.search(product_regex, url).group(0)
    
    stock_json = retrieve_stock(product_id)

    if stock_json['source'] == 2:
        # Stored under alternative product id
        alternative_product_id = fetch_alterantive_product_id(product_id)
        stock_json = retrieve_stock(alternative_product_id)
        
    return stock_json
            
    

In [101]:
get_stock_from_webpage('https://www.saq.com/fr/12073995')

{'stock': 627, 'source': 1}

In [102]:
get_stock_from_webpage('https://www.saq.com/fr/14376324')

{'stock': 156, 'source': 1}

In [ ]:
# 